# BigQuery

BigQuery is a NoSQL, column data structure, Data Warehouse. 

# Cost

In BigQuery, it doesn't cost you to create tables. It DOES, however, cost to store and query data. 
- Price per storage: https://cloud.google.com/bigquery/pricing#storage - it's approximately in the ballpark of 20 USD per terabyte per month


# Query optimisation

Optimising query cost:
- As it is a columnar database, `SELECT *` is the most expensive way to query data; so, **specify the exact columns you need** instead of using the wildcard `*` operator
- In partitioned and clustered tables, use `WHERE` on such columns
- Avoid query loops in BigQuery 

# Cluster and partition

https://cloud.google.com/bigquery/docs/clustered-tables

<img src="Media/cluster-partition.png">

In [ ]:
/*
Create a partitioned / clustered table
More info: https://cloud.google.com/bigquery/docs/creating-partitioned-tables#sql
*/
CREATE OR REPLACE TABLE temp.ez_temp1 (
    id int64,
    transaction_date DATE
)
PARTITION BY transaction_date
CLUSTER BY (id)
AS 
SELECT 1 AS id, CAST('2023-01-01' AS DATE) AS year
UNION ALL
SELECT 2 AS id, CAST('2023-02-01' AS DATE) AS year 

## Partitioning

> Basically splitting a table into smaller tables

Partitioning is dividing a table into smaller tables based on a single column (date or integer).

Examples:
- A table containing sales data could be partitioned by month using the column containing the sale date, or by geographic region using the state/province code;
- It is a good pratice to partition by date

Types of partitioning:
- Horizontal: assigning rows to different partitions;
- Vertical: assigning sets of columns to different partitions (must be done manually);

When partitioning a table horizontally, you must choose a **partition key** - a column whose values are used to assign a row to a particular partition. 

Partitioning methods:
- Range partitioning: 
  - break up tables by date ranges;
  - Allows for a maxvalue partition to catch any rows that don't map to any other partition
- List: 
  - allows you to specify which values will be assigned to each partition
  - Can be used with enumerated set of values, e.g. state codes (CA, TX, VA), currencies (USD, EUR, JPY)
  - doesn't provide for a spillover partition (to catch any rows that don't map to any other partition)
  - the column chosen as the partitioning key should contain only a small number of values
- Hash partitioning:
  - tries to distribute rows evenly across a set of partitions by applying a hashing function to the column value 
  - Hash partitioning works best when the partitioning key column contains a large number of distinct values
- Composite partitioning: 
  - Allows you to use two different types of partitioning for the same table

Check partitions - MySQL:

```sql
SELECT
    partition_name,
    partition_expression,
    partition_description
FROM information_schema.partitions
WHERE table_name = 'sales'
ORDER BY partition_ordinal_position;
```

Benefits:
- Partition pruning: You may only need to interact with as few as one partition instead of the entire table; 
- Partition-wise joins: if you execute a query that includes a join to a partitioned table and the query includes a condition on the partitioning column, the server can exclude any partitions that do not contain data pertinent to the query;
- An ability to delete data that is no longer needed, e.g. deleting a partition that is older than seven years old; 

Disadvantages:
- There is a hard number of partitions - A table cannot be partitioned into an infinite number of partitions

## Clustering

> Basically it's just sorting

Clustering: 
- Basically just sorting by values in up to 4 columns
- Improves performance of some queries with filter and aggregate clauses
- Better done when cardinality of values in a column or group of columns is large

## Sharding

Partition the data across multiple databases; 

for example, if you were to employ this strategy for a social media company, you might decide to implement 100 separate databases, each one hosting the data for approximately 10 million users. 